TF-IDF : rescale the frequency of words by how often they appear in all texts so that the scores for frequent words like 'the', that are also frequent across other text, get penalized.

In [1]:
from datetime import datetime, timedelta

In [2]:
FILE_PATH = '/Users/grace/mimic/csv/'

## sepsis 확진 시점 = sepsis 진단 시점 OR blood culture positive OR 처방

### 1. sepsis 진단

진단 시점이 여러번 일수 있어서 가장 빠른 admission 시점으로 진행하였습니다.

Two relevant timestamps are the admission and discharge times associated with the hospital stay ID ('hadm_id'). If you would like to find more specific time points associated with ICD-related events, one option would be to review the nursing notes.

In [3]:
df_diagnoses = pd.read_csv(FILE_PATH+'DIAGNOSES_ICD.csv', sep=',')
df_diagnoses[df_diagnoses.ICD9_CODE.isin(['77181', '99591', '99592', '67020', '67022', '67024'])].shape

(5409, 5)

In [4]:
df_sepsis_patients = df_diagnoses[df_diagnoses.ICD9_CODE.isin(['77181', '99591', '99592', '67020', '67022', '67024'])]
df_sepsis_patients.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
87,1547,117,164853,16.0,99592
144,1604,124,138376,6.0,99592
276,505,64,172056,3.0,99591
450,679,85,112077,18.0,99591
497,726,92,142807,2.0,77181


In [5]:
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

df_admissions = pd.read_csv(FILE_PATH+'admissions.csv', sep=',')
df_admissions.head()
df_admissions.ADMITTIME = df_admissions.ADMITTIME.apply(lambda x: datetime.strptime(x, DATETIME_FORMAT))
df_admissions.dtypes

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


ROW_ID                           int64
SUBJECT_ID                       int64
HADM_ID                          int64
ADMITTIME               datetime64[ns]
DISCHTIME                       object
DEATHTIME                       object
ADMISSION_TYPE                  object
ADMISSION_LOCATION              object
DISCHARGE_LOCATION              object
INSURANCE                       object
LANGUAGE                        object
RELIGION                        object
MARITAL_STATUS                  object
ETHNICITY                       object
EDREGTIME                       object
EDOUTTIME                       object
DIAGNOSIS                       object
HOSPITAL_EXPIRE_FLAG             int64
HAS_CHARTEVENTS_DATA             int64
dtype: object

In [6]:
df_sepsis_adm = pd.merge(df_sepsis_patients, df_admissions, on=['SUBJECT_ID', 'HADM_ID'])
df_sepsis_adm = df_sepsis_adm[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE',
                        'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
                        'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
                        'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 
                        'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']]

df_sepsis_adm.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,117,164853,16.0,99592,2133-11-13 21:34:00,2133-12-01 17:00:00,2133-12-01 17:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,NaN,NaN,ENCEPHALOPATHY;LIVER FAILURE,1,1
1,124,138376,6.0,99592,2165-12-31 18:55:00,2166-02-01 06:55:00,2166-02-01 06:55:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,NaN,NaN,PNEUMONIA;CHRONIC OBST PULM DISEASE,1,1
2,64,172056,3.0,99591,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,2143-03-03 03:46:00,2143-03-03 11:00:00,FUNGAL MENINGITIS,0,1
3,85,112077,18.0,99591,2167-07-25 18:49:00,2167-07-30 15:24:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,2167-07-25 16:37:00,2167-07-25 20:46:00,PNEUMONIA,0,1
4,92,142807,2.0,77181,2122-12-13 19:30:00,2123-03-04 13:47:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,NaN,NaN,NEWBORN,0,1


In [7]:
sepsis_standard = []
for i, grp in df_sepsis_adm.groupby('SUBJECT_ID'):

    first_diag = grp.sort_values(by='ADMITTIME').head(1)
    sepsis_standard.append({
        'SUBJECT_ID':i,
        'HADM_ID':first_diag['HADM_ID'].values[0],
        'SEQ_NUM':first_diag['SEQ_NUM'].values[0],
        'ICD9_CODE':first_diag['ICD9_CODE'].values[0],
        'STARTDATE':first_diag['ADMITTIME'].values[0],
        'ENDDATE':first_diag['DISCHTIME'].values[0],
        'ADMISSION_TYPE':first_diag['ADMISSION_TYPE'].values[0],
        'DIAGNOSIS':first_diag['DIAGNOSIS'].values[0]
    })

len(sepsis_standard)

4781

### 2. 처방 시점

In [8]:
SEPSIS_DRUGS = [
    'amikacin', 'amoxicillin', 'ampicillin', 'azithromycin', 'aztreonam', 'vancomycin', 
    'cefaclor', 'cefadroxil', 'cephalexin', 'cefamandole', 'cefepime', 'cefixime', 
    'cefot axime', 'cefoxitin', 'cefpodoxime', 'cefprozil', 'ceftazidime', 'ceftriaxone', 
    'cefur oxime', 'chloramphenicol', 'ciprofloxacin', 'clarithromycin', 'clindamycin', 
    'coli stin', 'dapsone', 'daptomycin', 'dicloxacillin', 'doripenem', 'doxycycline', 
    'ertapen em', 'erythromycin', 'ethambutol', 'flucloxacillin', 'fosfomycin', 'gatifloxacin', 
    'g eldanamycin', 'gentamicin', 'imipenem', 'isoniazid', 'kanamycin', 'levofloxacin', 
    'li nezolid', 'meropenem', 'methicillin', 'metronidazole', 'minocycline', 'moxifloxaci n', 
    'trovafloxacin', 'nafcillin', 'nalidixic acid', 'tobramycin', 'netilmicin', 'nitrofurantoin', 
    'norfloxacin', 'ofloxacin', 'par omomycin', 'penicillin', 'piperacillin', 'polymyxin', 
    'pyrazinamide', 'quinupristi n', 'rifabutin', 'rifampicin', 'rifampin', 'spectinomycin', 
    'streptomycin', 'sulfadia zine', 'sulfamethoxazole', 'telithromycin', 'tetracycline', 
    'ticarcillin', 'tigecy cline', 'trimethoprim'
]
# SEPSIS_DRUGS = [
#     'vancomycin', 'amikacin', 'gentamycin', 'tobramycin'
# ]
# ROUTES = ['OU','OS','OD','AU','AS','AD', 'TP']

In [9]:
df_prescriptions = pd.read_csv(FILE_PATH+'PRESCRIPTIONS.csv', sep=',')
df_prescriptions.dropna(subset=['STARTDATE'], inplace=True)
df_prescriptions.reset_index(drop=True)
# df_prescriptions.head()

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,4.690617e+08,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,5.601728e+07,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,5.482972e+07,20mg Tablet,20,mg,1,TAB,PO
5,2214774,6,107064,NaN,2175-06-11 00:00:00,2175-06-15 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF0,014198,5.601698e+07,Check with MD for Dose,1,dose,1,dose,PO
6,2215525,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
7,2216266,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
8,2215526,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
9,2214778,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF2,006561,5.601708e+07,2mg Tab,2,mg,1,TAB,PO


In [10]:
df_prescriptions.STARTDATE = df_prescriptions.STARTDATE.apply(lambda x: datetime.strptime(x, DATETIME_FORMAT))
len(df_prescriptions.query('DRUG.str.lower() in (@SEPSIS_DRUGS)')['SUBJECT_ID'].unique())

21116

In [11]:
for i, grp in df_prescriptions.query('DRUG.str.lower() in (@SEPSIS_DRUGS)').groupby('SUBJECT_ID'):
    first_diag = grp.sort_values(by='STARTDATE').head(1)
    sepsis_standard.append({
        'SUBJECT_ID':i,
        'HADM_ID':first_diag['HADM_ID'].values[0],
        'ROUTE':first_diag['ROUTE'].values[0],
        'DRUG':first_diag['DRUG'].values[0],
        'STARTDATE':first_diag['STARTDATE'].values[0],
        'ENDDATE':first_diag['ENDDATE'].values[0]
    })

### 3. Blood culture order가 있다면 sepsis의심 환자로 구분

In [12]:
# df_microbiology = pd.read_csv(FILE_PATH+'MICROBIOLOGYEVENTS.csv', sep=',')
# df_microbiology.head()
# df_microbiology.CHARTDATE = df_microbiology.CHARTDATE.apply(lambda x: datetime.strptime(x, DATETIME_FORMAT))

In [13]:
# BLOOD_CULTURE_STR = 'BLOOD CULTURE'

# for i, grp in df_microbiology.query('SPEC_TYPE_DESC==@BLOOD_CULTURE_STR').groupby('SUBJECT_ID'):
#     first_diag = grp.sort_values(by='CHARTTIME').head(1)
#     sepsis_standard.append({
#         'SUBJECT_ID':i,
#         'HADM_ID':first_diag['HADM_ID'].values[0],
#         'SPEC_TYPE_DESC':first_diag['SPEC_TYPE_DESC'].values[0],
#         'ORG_NAME':first_diag['ORG_NAME'].values[0],
#         'STARTDATE':first_diag['CHARTTIME'].values[0]
#     })

## 가장 빠른 sepsis 기준 시간 찾기

In [14]:
df_sepsis_standard = pd.DataFrame(sepsis_standard)
df_sepsis_standard.head()

,ADMISSION_TYPE,DIAGNOSIS,DRUG,ENDDATE,HADM_ID,ICD9_CODE,ROUTE,SEQ_NUM,STARTDATE,SUBJECT_ID
0,EMERGENCY,SEPSIS,NaN,2135-02-08 02:08:00,111970,99592,NaN,11.0,2135-01-30 20:50:00,21
1,EMERGENCY,ACUTE MYOCARDIAL INFARCTION-SEPSIS,NaN,2166-09-04 11:30:00,185910,99592,NaN,3.0,2166-08-10 00:28:00,38
2,EMERGENCY,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,NaN,2119-02-03 01:35:00,189535,99591,NaN,8.0,2119-01-04 18:12:00,61
3,EMERGENCY,"SEPSIS,URINARY TRACT INFECTION",NaN,2113-02-19 15:30:00,116009,99591,NaN,4.0,2113-02-15 00:19:00,62
4,EMERGENCY,FUNGAL MENINGITIS,NaN,2143-03-18 13:01:00,172056,99591,NaN,3.0,2143-03-03 09:25:00,64


In [15]:
df_sepsis_standard.dtypes

ADMISSION_TYPE            object
DIAGNOSIS                 object
DRUG                      object
ENDDATE                   object
HADM_ID                    int64
ICD9_CODE                 object
ROUTE                     object
SEQ_NUM                  float64
STARTDATE         datetime64[ns]
SUBJECT_ID                 int64
dtype: object

In [16]:
sepsis_earliest = []
for i, grp in df_sepsis_standard.groupby('SUBJECT_ID'):
    first_diag = grp.sort_values(by='STARTDATE').head(1)
    if first_diag['DRUG'].values[0] is not None:
        st_type = 'DRUG'
        st_detail = first_diag['DRUG'].values[0]
    elif first_diag['SPEC_TYPE_DESC'].values[0] is not None:
        st_type = 'BLOOD CULTURE'
        st_detail = first_diag['SPEC_TYPE_DESC'].values[0]
    else:
        st_type = 'DIAGNOSIS'
        st_detail = first_diag['DIAGNOSIS'].values[0]
        
    sepsis_earliest.append({
        'SUBJECT_ID':i,
        'HADM_ID':first_diag['HADM_ID'].values[0],
        'TYPE': st_type,
        'DETAIL': st_detail,
        'STARTDATE':first_diag['STARTDATE'].values[0],
        'ENDDATE':first_diag['ENDDATE'].values[0]
    })

In [17]:
df_sepsis_ear = pd.DataFrame(sepsis_earliest)
df_sepsis_ear.shape

(21512, 6)

In [18]:
len(df_sepsis_ear.SUBJECT_ID.unique())

21512

## 기준시간 48앞 부터 진단명 organize

In [19]:
df_dig_adm = pd.merge(df_diagnoses, df_admissions, on=['SUBJECT_ID', 'HADM_ID'])
df_dig_adm = df_dig_adm[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE',
                        'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
                        'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
                        'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 
                        'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']]
df_dig_adm.dtypes

SUBJECT_ID                       int64
HADM_ID                          int64
SEQ_NUM                        float64
ICD9_CODE                       object
ADMITTIME               datetime64[ns]
DISCHTIME                       object
DEATHTIME                       object
ADMISSION_TYPE                  object
ADMISSION_LOCATION              object
DISCHARGE_LOCATION              object
EDREGTIME                       object
EDOUTTIME                       object
DIAGNOSIS                       object
HOSPITAL_EXPIRE_FLAG             int64
HAS_CHARTEVENTS_DATA             int64
dtype: object

In [20]:
DURATION_DAY_UNIT = 0.5
DURATION_DAYS_BEFORE = 2
DURATION_DAYS_AFTER = 12
# 여러 입원이 중첩으로 날짜 기간이 매우 길어질수 있어서 일단 2주 [2틀- 진단 - 2틀 간격 12일] 의 기간으로 한정해서 진행했습니다.
#     df_dig_adm[df_dig_adm.SUBJECT_ID==i].sort_values(by='ADMITTIME').tail(1)

In [21]:
def generate_duration(start_time):
    time_lines = {}
    time_lines[1] = start_time
    time_lines[0] = start_time - timedelta(days=DURATION_DAY_UNIT)
    
    for k in range(-1, -int(DURATION_DAYS_BEFORE/DURATION_DAY_UNIT)-1, -1):
#         print(k)
        time_lines[k] = time_lines[k+1] - timedelta(days=DURATION_DAY_UNIT)
    
    for j in range(1, int(DURATION_DAYS_AFTER / DURATION_DAY_UNIT)+2):
#         print(j)
        time_lines[j] = time_lines[j-1] + timedelta(days=DURATION_DAY_UNIT)
    return time_lines

# def extract_information_by_time_standards(df, target_col, time_col, time_lines, row):
#     info_by_time = {}
#     info_by_time['SUBJECT_ID'] = row['SUBJECT_ID']
#     grp = df[df['SUBJECT_ID']==row['SUBJECT_ID']]
#     for k in sorted(time_lines.keys())[:len(time_lines.keys())-1]:
# #         print(k)
#         c_time = time_lines[k]
#         if c_time >= grp[time_col].min() and c_time <= grp[time_col].max():
#             info_by_time[k] = grp[(grp[time_col] >= time_lines[k]) & (df[time_col] < time_lines[k+1])][target_col].unique()
#     return info_by_time

def extract_information_by_time_standards(df, target_col, time_col, time_lines, row):
    info_by_time = {}
    info_by_time['SUBJECT_ID'] = row['SUBJECT_ID']
    for k in sorted(time_lines.keys())[:len(time_lines.keys())-1]:
#         print(k)
        grp = df[(df['SUBJECT_ID']==row['SUBJECT_ID']) & (df[time_col] >= time_lines[k]) & (df[time_col] < time_lines[k+1])]
        info_by_time[k] = grp[target_col].unique()
    return info_by_time

### 1. Diagnosis 진단은... 입원 시점과 날짜를 맞출수 밖에 없기 때문에 시간에 따른 분석이 어려움..

In [22]:
# for i, row in df_sepsis_ear[5:6].iterrows():
#     times = generate_duration(row['STARTDATE'])
#     print(times)
#     print(extract_information_by_time_standards(df_dig_adm, 'ICD9_CODE', 'ADMITTIME', times, row))

In [23]:
# import collections
# od = collections.OrderedDict(sorted(times.items()))
# od

### 2. lab

In [24]:
df_labs = pd.read_csv(FILE_PATH+'LABEVENTS.csv', sep=',')
df_labs.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [25]:
len(df_sepsis_ear.SUBJECT_ID.unique())

21512

In [26]:
df_labs = df_labs[df_labs.SUBJECT_ID.isin(df_sepsis_ear.SUBJECT_ID.unique())]
df_labs.shape

(20864056, 9)

In [27]:
df_labs.CHARTTIME = df_labs.CHARTTIME.apply(lambda x: datetime.strptime(x, DATETIME_FORMAT))

In [28]:
sepsis_ear_lab = []
for i, row in df_sepsis_ear.iterrows():
    if i%1000==0:
        print(i)
    times = generate_duration(row['STARTDATE'])
    sepsis_ear_lab.append(extract_information_by_time_standards(df_labs, 'ITEMID', 'CHARTTIME', times, row))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [29]:
df_sepsis_ear_lab = pd.DataFrame(sepsis_ear_lab)
df_sepsis_ear_lab.head()

,SUBJECT_ID,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,4,[],[],[],[],"[50861, 50863, 50868, 50878, 50882, 50883, 508...","[51463, 51464, 51466, 51476, 51478, 51484, 514...",[],"[50868, 50882, 50902, 50912, 50931, 50971, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50902, 50912, 50931, 50960, 509...",[],[],[],[],[],[],[],[],[],[],[]
1,9,[],[],[],"[50856, 50868, 50879, 50880, 50882, 50902, 509...","[50800, 50802, 50804, 50806, 50809, 50810, 508...","[50852, 50868, 50882, 50893, 50902, 50903, 509...","[50910, 50911, 51003, 51221, 51222, 51237, 512...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50893, 50960, 50971, 51221]","[50861, 50863, 50868, 50878, 50882, 50885, 508...","[51087, 51093, 50964]","[50862, 50868, 50882, 50893, 50902, 50912, 509...","[50800, 50802, 50804, 50818, 50820, 50821, 508...","[50819, 50820, 50821, 50826, 50827, 50908, 509...",[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
2,11,[],[],[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[51464, 51466, 51478, 51484, 51486, 51487, 514...",[],"[50868, 50882, 50893, 50902, 50912, 50920, 509...",[],[],[],[],[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50920, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[]
3,12,[],"[50861, 50863, 50867, 50868, 50878, 50882, 508...",[],[],[],"[51221, 51222, 51248, 51249, 51250, 51265, 512...","[50800, 50802, 50804, 50806, 50808, 50809, 508...","[50800, 50802, 50804, 50808, 50812, 50813, 508...","[51221, 51237, 51274, 51275, 50800, 50802, 508...","[50800, 50802, 50804, 50808, 50813, 50818, 508...","[50800, 50802, 50804, 50809, 50812, 50818, 508...","[50861, 50862, 50863, 50867, 50868, 50878, 508...","[50800, 50802, 50804, 50812, 50818, 50820, 508...","[50867, 50868, 50878, 50882, 50883, 50884, 508...","[50800, 50802, 50804, 50809, 50817, 50818, 508...","[50800, 50802, 50804, 50808, 50809, 50813, 508...","[50800, 50802, 50804, 50808, 50809, 50812, 508...","[50800, 50802, 50804, 50808, 50812, 50813, 508...","[50960, 50971]","[50800, 50802, 50804, 50808, 50813, 50818, 508...",[],"[50861, 50862, 50863, 50867, 50868, 50878, 508...",[],"[50861, 50863, 50868, 50878, 50882, 50885, 508...",[],"[50861, 50863, 50868, 50878, 50882, 50885, 508...",[],"[50861, 50863, 50868, 50878, 50882, 50885, 508...","[51463, 51464, 51466, 51476, 51478, 51484, 514..."
4,19,[],[],[],[],[],"[51484, 51486, 51487, 51491, 51492, 51493, 514...","[50893, 50960, 50970, 50993, 51003, 51237, 512...","[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]


In [30]:
df_sepsis_ear_lab.to_csv('./12h_sepsis_ear_labs.csv', sep=',', index=False)

In [32]:
df_sepsis_ear.shape

(21512, 6)

In [128]:
def jaccard_index(first_set, second_set):
    """ Computes jaccard index of two sets
        Arguments:
          first_set(set):
          second_set(set):
        Returns:
          index(float): Jaccard index between two sets; it is 
            between 0.0 and 1.0
    """
    # If both sets are empty, jaccard index is defined to be 1
    index = 1.0
    if first_set or second_set:
        index = (float(len(first_set.intersection(second_set))) 
             / len(first_set.union(second_set)))

    return index

In [34]:
# test = pd.read_csv('./sepsis_ear_labs.csv', sep=',')
# test.head()
seed_patient = df_sepsis_ear_lab[df_sepsis_ear_lab.SUBJECT_ID==25030]
seed_patient

,SUBJECT_ID,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
8927,25030,"[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[51237, 51274, 51275, 50800, 50802, 50804, 508...","[51237, 51274, 51275, 50800, 50802, 50804, 508...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50800, 50802, 50804, 50808, 50818, 50820, 508...","[50967, 50800, 50802, 50804, 50818, 50820, 508...","[50800, 50802, 50804, 50808, 50813, 50818, 508...","[50902, 50912, 50931, 50960, 50970, 50971, 509...","[50800, 50802, 50804, 50818, 50820, 50821, 508...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50800, 50802, 50804, 50816, 50818, 50819, 508...","[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50800, 50802, 50804, 50810, 50811, 50818, 508...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[51351, 51355, 51360, 51362, 51363, 50861, 508...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50861, 50862, 50863, 50867, 50868, 50878, 508...",[50931],"[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50983, 51009]","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[51351, 51355, 51360, 51362, 51363, 50800, 508..."


In [ ]:
# df_sepsis_ear_lab = df_sepsis_ear_lab[df_sepsis_ear_lab.SUBJECT_ID!=25030]

In [ ]:
# seed_patient = {}
# seed_patient['0'] = [51301, 50813, 50861, 50867, 50868, 50878, 50882, 50885, 50893, 50902, 50910, 50911, 50912, 50931, 50956, 50960, 50971, 50983, 51003, 51006, 51137, 51143, 51144, 51146, 51200, 51221, 51222, 51233, 51237, 51244, \
#                      51246, 51248, 51249, 51250, 51251, 51252, 51254, 51255, 51256, \
#                      51265, 51267, 51268, 51274, 51275, 51277, 51279, 51294]
# seed_patient['1'] = [50868, 50882, 50893, 50902, 50909, 50931, 50954, 50960, 50970, 50971, 50983, 51221, 50800, 50802, 50804, 50812, 50813, 50817, 50818, 50820, 50821, 50825, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301, 51237, 51274, 51275, 50910, 50911, 50912, 51003, 51006, 51009]
# seed_patient['2'] = [50868, 50882, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51233, 51237, 51244, 51246, 51248, 51249, 51250, 51254, 51256, 51265, 51267, 51274, 51275, 51277, 51279, 51301, 50852]
# seed_patient['3'] = [50868, 50882, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301]

In [75]:
cols = df_sepsis_ear_lab.columns[1:]
cols

Index(['-4', '-3', '-2', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24'],
      dtype='object')

In [129]:
# df_sepsis_ear_lab.head()

In [131]:
for col in range(-4, 25):#cols:
    print(col)
#     print(type(col))
    df_sepsis_ear_lab[str(col)+'_jaccard'] = df_sepsis_ear_lab[str(col)].apply(lambda x: jaccard_index(set(seed_patient[0].values[0]), set(x)))

df_sepsis_ear_lab.head()

-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


,SUBJECT_ID,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,...,10_jaccard,11_jaccard,12_jaccard,13_jaccard,14_jaccard,15_jaccard,16_jaccard,17_jaccard,18_jaccard,19_jaccard,20_jaccard,21_jaccard,22_jaccard,23_jaccard,24_jaccard
0,4,[],[],[],[],"[50861, 50863, 50868, 50878, 50882, 50883, 508...","[51463, 51464, 51466, 51476, 51478, 51484, 514...",[],"[50868, 50882, 50902, 50912, 50931, 50971, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50868, 50882, 50893, 50902, 50912, 50931, 509...",...,0.000000,0.689655,0.000000,0.310345,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,9,[],[],[],"[50856, 50868, 50879, 50880, 50882, 50902, 509...","[50800, 50802, 50804, 50806, 50809, 50810, 508...","[50852, 50868, 50882, 50893, 50902, 50903, 509...","[50910, 50911, 51003, 51221, 51222, 51237, 512...","[50868, 50882, 50893, 50902, 50912, 50931, 509...","[50893, 50960, 50971, 51221]","[50861, 50863, 50868, 50878, 50882, 50885, 508...","[51087, 51093, 50964]","[50862, 50868, 50882, 50893, 50902, 50912, 509...","[50800, 50802, 50804, 50818, 50820, 50821, 508...","[50819, 50820, 50821, 50826, 50827, 50908, 509...",...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,11,[],[],[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[51464, 51466, 51478, 51484, 51486, 51487, 514...",[],"[50868, 50882, 50893, 50902, 50912, 50920, 509...",[],[],[],[],[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",...,0.000000,0.655172,0.000000,0.689655,0.000000,0.689655,0.0,0.666667,0.0,0.689655,0.0,0.689655,0.0,0.689655,0.0
3,12,[],"[50861, 50863, 50867, 50868, 50878, 50882, 508...",[],[],[],"[51221, 51222, 51248, 51249, 51250, 51265, 512...","[50800, 50802, 50804, 50806, 50808, 50809, 508...","[50800, 50802, 50804, 50808, 50812, 50813, 508...","[51221, 51237, 51274, 51275, 50800, 50802, 508...","[50800, 50802, 50804, 50808, 50813, 50818, 508...","[50800, 50802, 50804, 50809, 50812, 50818, 508...","[50861, 50862, 50863, 50867, 50868, 50878, 508...","[50800, 50802, 50804, 50812, 50818, 50820, 508...","[50867, 50868, 50878, 50882, 50883, 50884, 508...",...,0.193548,0.527273,0.176471,0.426230,0.068966,0.531915,0.0,0.555556,0.0,0.606061,0.0,0.606061,0.0,0.606061,0.0
4,19,[],[],[],[],[],"[51484, 51486, 51487, 51491, 51492, 51493, 514...","[50893, 50960, 50970, 50993, 51003, 51237, 512...","[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],"[50868, 50882, 50893, 50902, 50912, 50931, 509...",[],[],...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [151]:
df_sepsis_ear_lab[[str(i)+'_jaccard' for i in cols]].sort_values(by='-4_jaccard', ascending=False).head()

,-4_jaccard,-3_jaccard,-2_jaccard,-1_jaccard,0_jaccard,1_jaccard,2_jaccard,3_jaccard,4_jaccard,5_jaccard,6_jaccard,7_jaccard,8_jaccard,9_jaccard,10_jaccard,11_jaccard,12_jaccard,13_jaccard,14_jaccard,15_jaccard,16_jaccard,17_jaccard,18_jaccard,19_jaccard,20_jaccard,21_jaccard,22_jaccard,23_jaccard,24_jaccard
3431,0.966667,0.575000,0.557692,0.783784,0.369565,0.722222,0.333333,0.512821,0.250000,0.666667,0.066667,0.540541,0.866667,0.763158,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
16840,0.965517,0.818182,0.000000,0.965517,0.000000,0.793103,0.000000,0.793103,0.000000,0.666667,0.000000,0.450980,0.000000,0.793103,0.37931,0.793103,0.0,0.793103,0.0,0.657143,0.0,0.638889,0.0,0.0,0.0,0.657143,0.0,0.605263,0.0
14894,0.965517,0.733333,0.000000,0.666667,0.000000,0.416667,0.000000,0.526316,0.000000,0.689655,0.000000,0.689655,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
20218,0.935484,0.793103,0.000000,0.689655,0.413793,0.793103,0.187500,0.689655,0.000000,0.689655,0.000000,0.689655,0.000000,0.666667,0.00000,0.689655,0.0,0.000000,0.0,0.689655,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
9878,0.935484,0.212121,0.093750,0.644444,0.333333,1.000000,0.000000,0.966667,0.181818,0.700000,0.187500,0.933333,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [181]:
df_sepsis_ear_lab[[str(i)+'_jaccard' for i in cols]][df_sepsis_ear_lab['-4_jaccard']>.8].shape

(60, 29)

In [167]:
pd.set_option('display.max_colwidth', -1)

In [176]:
seed_patient[-4].values[0].sort()
seed_patient[-4].values[0]

array([50868, 50882, 50893, 50902, 50912, 50931, 50960, 50970, 50971,
       50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265,
       51274, 51275, 51277, 51279, 51301])

In [178]:
df_sepsis_ear_lab.loc[3431, '-4']

array([50800, 50802, 50804, 50808, 50818, 50820, 50821, 50868, 50882,
       50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006,
       51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275,
       51277, 51279, 51301])